In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Initialize the Selenium driver
def init_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def extract_data(url, retries=3):
    driver = init_driver()
    for attempt in range(retries):
        try:
            driver.get(url)
            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'post-detail'))
            )
            # Click the element to trigger the modal
            alert_trigger = driver.find_element(By.CLASS_NAME, 'post-detail')
            alert_trigger.click()

            WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CLASS_NAME, 'data-box'))
            )

            time.sleep(2) 

            # Extract data
            data = {}
            try:
                data['address'] = driver.find_element(By.CSS_SELECTOR, '.contact-details-box .address-details-m h2 + p').text
            except Exception as e:
                data['address'] = f"Error extracting address: {e}"

            try:
                data['telephone'] = driver.find_element(By.XPATH, "//div[@class='address-details-m'][2]/p").text
            except Exception as e:
                data['telephone'] = f"Error extracting telephone: {e}"

            try:
                data['email'] = driver.find_element(By.XPATH, "//div[@class='address-details-m'][3]//a").text
            except Exception as e:
                data['email'] = f"Error extracting email: {e}"

            try:
                image_element = driver.find_element(By.CLASS_NAME, 'left-counsellor-profile').find_element(By.TAG_NAME, 'img')
                data['image_url'] = image_element.get_attribute("src")
            except Exception as e:
                data['image_url'] = f"Error extracting image URL: {e}"

            driver.quit()
            return data

        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            driver.quit()
            if attempt < retries - 1:
                time.sleep(5)
            else:
                raise e

url = "https://www.icef.com/academy/qualified-education-agent-counsellors-qeacs-list/?fbclid=IwY2xjawFBFxxleHRuA2FlbQIxMAABHVdkkFJ39b_JWTzncBLUzWZEEtObmSmNqYRX3OsNKoIPHS8scEwJEiHldA_aem_nlquSyxhhRRwjftVvDdswg"

# Extract and save data in csv file
"""data = extract_data(url)
if data:
    df = pd.DataFrame([data])
    df.to_csv('extracted_data.csv', mode='a', header=False, index=False)
    print("Data saved to CSV.")"""

# Extract data and print it
data = extract_data(url)
if data:
    print("Extracted Data:")
    for key, value in data.items():
        print(f"{key}: {value}")
